In [ ]:
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt

### Loading Data

In [ ]:
admissions = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/admissions.csv")
print(admissions.head(200))

In [ ]:
'''patients = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/patients.csv")
print(patients.head(200))'''

In [ ]:
drgcodes = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/drgcodes.csv")
print(drgcodes.head(200))

In [ ]:
procedures_icd = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/procedures_icd.csv")
print(procedures_icd.head(200))

In [ ]:
diagnoses_icd = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/diagnoses_icd.csv")
print(diagnoses_icd.head(200))

#### Look at Different Categorical Feattures

In [ ]:
'''# Count unique values in admissions
sql_query = """
SELECT COUNT(DISTINCT subject_id) as patients, COUNT(DISTINCT hadm_id) as stays,COUNT(DISTINCT admission_type) as type,COUNT(DISTINCT admit_provider_id) as provider, COUNT(DISTINCT admission_location) as admission_location, COUNT(DISTINCT discharge_location) as discharge_location, COUNT(DISTINCT insurance) as insurance, COUNT(DISTINCT language) as language, COUNT(DISTINCT marital_status) as marital_status, COUNT(DISTINCT race) as race
FROM admissions


"""
print(ps.sqldf(sql_query,locals()))'''


In [ ]:
'''sql_query = """
SELECT COUNT(DISTINCT gender ) AS gender, COUNT(DISTINCT anchor_age) as anchor_age, COUNT(DISTINCT anchor_year ) AS anchor_year,COUNT(DISTINCT anchor_year_group) AS anchor_year_group, COUNT(DISTINCT dod ) AS dod
FROM patients


"""
print(ps.sqldf(sql_query,locals()))'''


In [ ]:
'''# drgcodes
sql_query = """
SELECT COUNT(DISTINCT drg_type) AS drg_typeCategories , COUNT(DISTINCT drg_code) AS drg_codeCategories, COUNT(DISTINCT drg_severity) AS drg_severityCategories , COUNT(DISTINCT drg_mortality) AS drg_mortalityCategories
FROM drgcodes


"""
print(ps.sqldf(sql_query,locals()))'''

In [ ]:
'''sql_query = """
SELECT COUNT(DISTINCT icd_code) AS icd_code, COUNT(DISTINCT icd_version) AS icd_version
FROM procedures_icd


"""
print(ps.sqldf(sql_query,locals()))'''




In [ ]:
'''sql_query = """
SELECT COUNT(DISTINCT icd_code) AS icd_code, COUNT(DISTINCT icd_version) AS icd_version
FROM diagnoses_icd


"""
print(ps.sqldf(sql_query,locals()))'''


In [ ]:
# selecting columns for feature eng. and further analysis
sql_query = """
SELECT
    subject_id,
    hadm_id,
    admittime,
    dischtime,
    admission_type,
    admission_location,
    insurance
FROM admissions
"""

# Load the admissions data
admissions = ps.sqldf(sql_query, locals())

In [ ]:
# convert admit time and dischtime to datetime format
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])

In [ ]:
# calculating Length of stays LOS in days
# number of days patient stayed in the hospital
admissions['length_of_stay'] = (admissions['dischtime'] - admissions['admittime']).dt.total_seconds() / (24 * 3600)
admissions.head(200)

In [ ]:
# encode admission type
admissions['admission_type_encoded'] = admissions['admission_type'].astype('category').cat.codes
admissions.head(200)

In [ ]:
# Creating a lookup table to keep track of my features
admission_type_lookup = pd.DataFrame({
    'admission_type': admissions['admission_type'].astype('category').cat.categories,
    'admission_type_encoded': range(len(admissions['admission_type'].astype('category').cat.categories))
})

admission_type_lookup.head(200)

Admission location

In [ ]:
# admission location encoding
admissions['admission_location_encoded'] = admissions['admission_location'].astype('category').cat.codes
admissions.head(200)

In [ ]:
# create lookup table for refrence
admission_location_lookup = pd.DataFrame({
    'category': 'admission_location',
    'value': admissions['admission_location'].astype('category').cat.categories,
    'encoded_value': range(len(admissions['admission_location'].astype('category').cat.categories))
})
admission_location_lookup.head(200)

In [ ]:
# encode insurance
admissions['insurance_encoded'] = admissions['insurance'].astype('category').cat.codes

In [ ]:
# creating lookup table for reference
insurance_lookup = pd.DataFrame({
    'category': 'insurance',
    'value': admissions['insurance'].astype('category').cat.categories,
    'encoded_value': range(len(admissions['insurance'].astype('category').cat.categories))
})
insurance_lookup.head(200)

In [ ]:
# 1. common structure for look up table
admission_type_lookup['category'] = 'admission_type'
admission_type_lookup = admission_type_lookup.rename(columns={'admission_type': 'value', 'admission_type_encoded': 'encoded_value'})

# admissions
admission_location_lookup['category'] = 'admission_location'
admission_location_lookup = admission_location_lookup.rename(columns={'admission_location': 'value', 'admission_location_encoded': 'encoded_value'})

# Insurance
insurance_lookup['category'] = 'insurance'
insurance_lookup = insurance_lookup.rename(columns={'insurance': 'value', 'insurance_encoded': 'encoded_value'})

# 2 all look up tables into one structure
lookup_combined = pd.concat([admission_type_lookup, admission_location_lookup, insurance_lookup], ignore_index=True)

#  3: Rearrange columns
lookup_combined = lookup_combined[['category', 'value', 'encoded_value']]
lookup_combined_admissions = lookup_combined.sort_values(by=['category', 'encoded_value']).reset_index(drop=True)

# save output
lookup_combined_admissions.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/AdmissionsLU.csv', index=False)

In [ ]:
# aggregate features from admissions for one patient ( multiple stays - multiple entries)
admissions_aggregated = admissions.groupby('subject_id').agg({
    'length_of_stay': ['mean', 'sum', 'max', 'count'],  # Length of stay stats
    'admission_type_encoded': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # mode   admission type
    'admission_location_encoded': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # mode  admission location
    'insurance_encoded': lambda x: x.mode().iloc[0] if not x.mode().empty else None  # mode insurance type
}).reset_index()

# Rename columns
admissions_aggregated.columns = [
    'subject_id',
    'mean_los', 'total_los', 'max_los', 'num_admissions',
    'most_common_admission_type',
    'most_common_admission_location',
    'most_common_insurance_type'
]

In [ ]:
# save into new data frame for clarity
final_admissions_features = admissions_aggregated.copy()

In [ ]:
final_admissions_features.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/final_admission_features.csv', index=False)

In [ ]:
p = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/patients.csv")
# really strange bug --> got admission data for patients object --> load patients df again.
#extracting all relevant features
ql_query = """
SELECT
    subject_id,
    gender,
    anchor_age,
    dod
FROM p
"""


patients = ps.sqldf(sql_query, locals())
patients = p

In [ ]:
'''sql_query = """
SELECT
    COUNT( subject_id)
FROM patients
"""

print(ps.sqldf(sql_query, locals()))'''

In [ ]:
patients.head(200)

In [ ]:
# Binary encoding gender
patients['gender_encoded'] = patients['gender'].astype('category').cat.codes

# Gender Lookup Table
gender_lookup = pd.DataFrame({
    'category': 'gender',
    'value': patients['gender'].astype('category').cat.categories,
    'encoded_value': range(len(patients['gender'].astype('category').cat.categories))
})

In [ ]:
gender_lookup.head(200)

In [ ]:
# function to bucketize age
def bucketize_age(age):
    if age <= 20:
        return 0
    elif age <= 40:
        return 1
    elif age <= 60:
        return 2
    elif age <= 80:
        return 3
    else:
        return 4

# apply it to dataframe
patients['age_bucket'] = patients['anchor_age'].apply(bucketize_age)

# Age Bucket Lookup Table
age_bucket_lookup = pd.DataFrame({
    'category': 'age_bucket',
    'value': ['0-20', '21-40', '41-60', '61-80', '81+'],
    'encoded_value': range(5)
})



In [ ]:
age_bucket_lookup.head(200)

In [ ]:
# if dod is not null 1 else 0 --> this means if the patient died in hospital or up to one year after the stay it is marked as died in my data
patients['mortality'] = patients['dod'].notnull().astype(int)

# Mortality Lookup Table
mortality_lookup = pd.DataFrame({
    'category': 'mortality',
    'value': ['Survived', 'Died'],
    'encoded_value': [0, 1]
})



In [ ]:
mortality_lookup.head(200)

In [ ]:
# 1 add category lables
gender_lookup['category'] = 'gender'
age_bucket_lookup['category'] = 'age_bucket'
mortality_lookup['category'] = 'mortality'

# 2 combine all look up tables into one
lookup_combined_patients = pd.concat([gender_lookup, age_bucket_lookup, mortality_lookup], ignore_index=True)

# 3 rearange and sort
lookup_combined_patients = lookup_combined_patients[['category', 'value', 'encoded_value']]
lookup_combined_patients = lookup_combined_patients.sort_values(by=['category', 'encoded_value']).reset_index(drop=True)

# Save Lookup Table
lookup_combined_patients.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/patients_lookup_table.csv', index=False)
lookup_combined_patients.head(200)

In [ ]:
# keep only required columns
patients_final = patients[['subject_id', 'gender_encoded', 'age_bucket', 'mortality']]

# save final data frame
patients_final.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/final_patients_features.csv', index=False)


patients_final.head(200)

In [ ]:
# selecting data
sql_query = """
SELECT
    subject_id,
    hadm_id,
    icd_code,
    icd_version
FROM procedures_icd
"""


procedures = ps.sqldf(sql_query, locals())

In [ ]:
# frequency of all icd codes across all admissions
icd_counts = procedures.groupby(['icd_code', 'icd_version']).size().reset_index(name='count')
icd_counts = icd_counts.sort_values(by='count', ascending=False)

In [ ]:
# extracting most common 1k icd codes
top_icd_codes = icd_counts.head(1000)
top_icd_codes_list = top_icd_codes['icd_code'].tolist()

In [ ]:
# binary matrix for top icd codes
# each column shows if a icd code is present for a procedure
icd_binary_matrix = pd.DataFrame({
    f'icd_code_{code}': (procedures['icd_code'] == code).astype(int)
    for code in top_icd_codes_list
})

# Concatenate binary columns to original data frame
procedures = pd.concat([procedures, icd_binary_matrix], axis=1)

In [ ]:
# Step 5: Aggregate binary icd codes by subject id.
binary_icd_columns = [col for col in procedures.columns if col.startswith('icd_code_')]

# A group by subject id
aggregated_features = procedures.groupby('subject_id')[binary_icd_columns].sum().reset_index()

# total icd code --> new feature
aggregated_features['total_icd_codes'] = aggregated_features[binary_icd_columns].sum(axis=1)



In [ ]:
# save final state
final_icd_features = aggregated_features.copy()
final_icd_features.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/final_icd_procedures_features.csv', index=False)


In [ ]:
final_icd_features.head(200)

In [ ]:
# extract all relevant features
sql_query = """
SELECT
    subject_id,
    hadm_id,
    icd_code,
    icd_version
FROM diagnoses_icd
"""
diagnoses = ps.sqldf(sql_query, locals())

In [ ]:
diagnoses.head(200)

In [ ]:
# frequency of all icd codes across dataset
icd_counts = diagnoses.groupby(['icd_code', 'icd_version']).size().reset_index(name='count')
icd_counts = icd_counts.sort_values(by='count', ascending=False)


In [ ]:
# top most 1k icd codes
top_icd_codes = icd_counts.head(1000)
# transforming into list
top_icd_codes_list = top_icd_codes['icd_code'].tolist()


In [ ]:
# binary matrix for features
icd_binary_matrix = pd.DataFrame({
    f'icd_code_{code}': (diagnoses['icd_code'] == code).astype(int)
    for code in top_icd_codes_list
})

In [ ]:
# concatenate binary columns
diagnoses = pd.concat([diagnoses, icd_binary_matrix], axis=1)


In [ ]:

# aggregate features by subject_id
binary_icd_columns = [col for col in diagnoses.columns if col.startswith('icd_code_')]
# group by subject id
aggregated_features = diagnoses.groupby('subject_id')[binary_icd_columns].sum().reset_index()
# new feature --> total icd codes
aggregated_features['total_icd_codes'] = aggregated_features[binary_icd_columns].sum(axis=1)


save final processed data

In [ ]:
# save final state
final_icd_diagnoses_features = aggregated_features.copy()
final_icd_diagnoses_features.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/final_icd_diagnoses_features.csv', index=False)


final_icd_diagnoses_features.head(200)

In [ ]:
'''sql_query = """
SELECT
    COUNT(drg_type)
FROM drgcodes
    WHERE drg_type = 'APR'

"""

print(ps.sqldf(sql_query, locals()))'''

In [ ]:
'''sql_query = """
SELECT
    drg_code
FROM drgcodes


"""

print(ps.sqldf(sql_query, locals()))'''

In [ ]:
'''sql_query = """
SELECT
    drg_code,
    COUNT(*) AS count
FROM drgcodes
GROUP BY drg_code
ORDER BY count DESC
LIMIT 100;
"""

print(ps.sqldf(sql_query, locals()))'''

 Loading Data  (sql)

In [ ]:
sql_query = """
SELECT
    subject_id,
    hadm_id,
    drg_code,
    drg_severity,
    drg_mortality
FROM drgcodes
"""

# Load data
drgcodes = ps.sqldf(sql_query, locals())

In [ ]:
# Calculate frequency of drg codes across data set
drg_code_counts = drgcodes.groupby('drg_code').size().reset_index(name='count')
drg_code_counts = drg_code_counts.sort_values(by='count', ascending=False)

# Select top most hundred drg codes
top_drg_codes = drg_code_counts.head(100)
top_drg_codes_list = top_drg_codes['drg_code'].tolist()
print(top_drg_codes_list)

In [ ]:
# binary  column for drg codes 1 present for subject, 0 absent
drg_binary_matrix = pd.DataFrame({
    f'drg_code_{code}': (drgcodes['drg_code'] == code).astype(int)
    for code in top_drg_codes_list
})

# concatenate binary features.
drgcodes = pd.concat([drgcodes, drg_binary_matrix], axis=1) # adding it to the original dataframe

In [ ]:
# binary drg column
binary_drg_columns = [col for col in drgcodes.columns if col.startswith('drg_code_')]

# Aggregate binary column and sum to get total count
aggregated_drg_features = drgcodes.groupby('subject_id')[binary_drg_columns].sum().reset_index()


# mean, maximum, and mode for  severity and mortality risk levels
aggregated_drg_features['mean_drg_severity'] = drgcodes.groupby('subject_id')['drg_severity'].mean().reset_index(drop=True)

aggregated_drg_features['max_drg_severity'] = drgcodes.groupby('subject_id')['drg_severity'].max().reset_index(drop=True)

aggregated_drg_features['mode_drg_severity'] = drgcodes.groupby('subject_id')['drg_severity'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index(drop=True)

aggregated_drg_features['mean_drg_mortality'] = drgcodes.groupby('subject_id')['drg_mortality'].mean().reset_index(drop=True)

aggregated_drg_features['max_drg_mortality'] = drgcodes.groupby('subject_id')['drg_mortality'].max().reset_index(drop=True)

aggregated_drg_features['mode_drg_mortality'] = drgcodes.groupby('subject_id')['drg_mortality'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index(drop=True)


In [ ]:
# Save final drg codes
final_drg_features = aggregated_drg_features.copy()
final_drg_features.to_csv('/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/final_drg_features.csv', index=False)


final_drg_features.head(200)


In [ ]:
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'

# loading data
final_patients_features = pd.read_csv(f"{path}final_patients_features.csv")
final_admission_features = pd.read_csv(f"{path}final_admission_features.csv")
final_icd_procedures_features = pd.read_csv(f"{path}final_icd_procedures_features.csv")
final_icd_diagnoses_features = pd.read_csv(f"{path}final_icd_diagnoses_features.csv")
final_drg_features = pd.read_csv(f"{path}final_drg_features.csv")


In [ ]:
# inner join to get subject ids of patients that are in all tables
sql_query = """
SELECT
    p.subject_id
FROM
    final_patients_features p
INNER JOIN
    final_admission_features a
ON
    p.subject_id = a.subject_id
INNER JOIN
    final_icd_procedures_features c
ON
    p.subject_id = c.subject_id
INNER JOIN
    final_icd_diagnoses_features d
ON
    p.subject_id = d.subject_id
INNER JOIN
    final_drg_features r
ON
    p.subject_id = r.subject_id
"""

subject_ids_in_all_tables = ps.sqldf(sql_query, locals())


In [ ]:
# saving list of all patients
subject_ids_in_all_tables.to_csv(f"{path}final_subject_ids_in_all_tables.csv", index=False)

In [ ]:
# num of patients present in all tables
sql_query = """
SELECT
Count(DISTINCT subject_id)
FROM
subject_ids_in_all_tables """

print(ps.sqldf(sql_query, locals()))

In [ ]:
# total num of patients in all tables
total_patients = len(final_patients_features)
patients_in_all_tables = len(subject_ids_in_all_tables)

# percentage patients in all tables vs missing in any table
present_percentage = (patients_in_all_tables / total_patients) * 100
missing_percentage = 100 - present_percentage

# visualize data
summary_data = pd.DataFrame({
    "Category": ["Patients in All Tables", "Missing from Any Table"],
    "Count": [patients_in_all_tables, total_patients - patients_in_all_tables],
    "Percentage": [present_percentage, missing_percentage]
})

# Ploting
plt.figure(figsize=(8, 5))
plt.bar(summary_data["Category"], summary_data["Percentage"], color=["blue", "orange"], alpha=0.7)

# percentage labels
for i, percentage in enumerate(summary_data["Percentage"]):
    plt.text(i, percentage + 2, f"{percentage:.1f}%", ha='center', va='bottom', fontsize=12)

# title and label
plt.title("Patients Present in All Tables vs Missing", fontsize=14)
plt.ylabel("Percentage", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display
plt.tight_layout()
plt.show()

In [ ]:
present_percentage = (patients_in_all_tables / total_patients) * 100
missing_percentage = 100 - present_percentage

# Print  percentages
print(f"Present Percentage: {present_percentage:.2f}%")
print(f"Missing Percentage: {missing_percentage:.2f}%")

In [ ]:
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'

# Load data
final_patients_features = pd.read_csv(f"{path}final_patients_features.csv")
final_admission_features = pd.read_csv(f"{path}final_admission_features.csv")
final_icd_procedures_features = pd.read_csv(f"{path}final_icd_procedures_features.csv")
final_icd_diagnoses_features = pd.read_csv(f"{path}final_icd_diagnoses_features.csv")
final_drg_features = pd.read_csv(f"{path}final_drg_features.csv")

In [ ]:
# saving output
subject_with_all_the_data_expect_procedures.to_csv(f"{path}allpatients_expect_procedures.csv", index=False)

In [ ]:
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'
# load data
subject_with_all_the_data_expect_procedures = pd.read_csv(f"{path}allpatients_expect_procedures.csv")
# count subject ids with missing entries --> 5311 missing
sql_query = """
SELECT
    COUNT(subject_id) AS missing_count
FROM
    subject_with_all_the_data_expect_procedures
WHERE
    subject_id IS NULL OR
    gender_encoded IS NULL OR
    age_bucket IS NULL OR
    mortality IS NULL OR
    mean_los IS NULL OR
    total_los IS NULL OR
    max_los IS NULL OR
    num_admissions IS NULL OR
    most_common_admission_type IS NULL OR
    most_common_admission_location IS NULL OR
    most_common_insurance_type IS NULL OR
    mean_drg_severity IS NULL OR
    max_drg_severity IS NULL OR
    mode_drg_severity IS NULL OR
    mean_drg_mortality IS NULL OR
    mode_drg_mortality IS NULL OR
    totalDiagnosesIcdCodes IS NULL
"""
print(ps.sqldf(sql_query, locals()))

In [ ]:
# filtering out rows with null values
sql_query = """
SELECT
    *
FROM
    subject_with_all_the_data_expect_procedures
WHERE
    subject_id IS NOT NULL AND
    gender_encoded IS NOT NULL AND
    age_bucket IS NOT NULL AND
    mortality IS NOT NULL AND
    mean_los IS NOT NULL AND
    total_los IS NOT NULL AND
    max_los IS NOT NULL AND
    num_admissions IS NOT NULL AND
    most_common_admission_type IS NOT NULL AND
    most_common_admission_location IS NOT NULL AND
    most_common_insurance_type IS NOT NULL AND
    mean_drg_severity IS NOT NULL AND
    max_drg_severity IS NOT NULL AND
    mode_drg_severity IS NOT NULL AND
    mean_drg_mortality IS NOT NULL AND
    mode_drg_mortality IS NOT NULL AND
    totalDiagnosesIcdCodes IS NOT NULL
"""


filtered_data = ps.sqldf(sql_query, locals())

# Overwrite original df with  filtered data (rows with no NULL values)

subject_with_all_the_data_expect_procedures.head(100)

In [ ]:
# save filtered data
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'
filtered_data.to_csv(f"{path}cleaned_allpatients_expect_procedures.csv", index=False)


In [ ]:
sql_query = """
SELECT
COUNT(mortality)
FROM
   subject_with_all_the_data_expect_procedures
   WhERE mortality = 0
    """
print(ps.sqldf(sql_query, locals()))

In [ ]:
sql_query = """
SELECT
COUNT(mortality)
FROM
   subject_with_all_the_data_expect_procedures
   WhERE mortality = 1
    """
print(ps.sqldf(sql_query, locals()))

In [ ]:
# get patients that died
sql_query = """
SELECT
*
FROM
   subject_with_all_the_data_expect_procedures
   WhERE mortality = 1
    """
mort1 = ps.sqldf(sql_query, locals())


In [ ]:
# Select patients that survived
sql_query = """
SELECT
*
FROM
   subject_with_all_the_data_expect_procedures
   WhERE mortality = 0
    """
mort0 = ps.sqldf(sql_query, locals())


In [ ]:
# Randomly sample majority class
# hardcoded value here but with length n --> dynamic
mort0_sampled = mort0.sample(n=35134, random_state=31)

In [ ]:
# Combine datasets to get a balanced df
balanced_dataset = pd.concat([mort1, mort0_sampled])

# shuffle dataset --> in order so i dont have to do it before fitting...
balanced_dataset = balanced_dataset.sample(frac=1, random_state=55).reset_index(drop=True)

In [ ]:
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'
balanced_dataset.to_csv(f"{path}cleaned_and_balanced_allpatients_expect_procedures.csv", index=False)

In [ ]:
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'
file_name = "cleaned_and_balanced_allpatients_expect_procedures.csv"
dataset = pd.read_csv(f"{path}{file_name}")
print(dataset.info())

In [ ]:
from sklearn.preprocessing import StandardScaler

# List of Columns to scale
columns_to_scale = [
    "mean_los", "total_los", "max_los",
    "num_admissions",
    "mean_drg_severity", "max_drg_severity", "mode_drg_severity",
    "mean_drg_mortality", "max_drg_mortality", "mode_drg_mortality",
    "totalDiagnosesIcdCodes"
]


# standard scaler
scaler = StandardScaler()

# scaling of the columns that are listed above
dataset[columns_to_scale] = scaler.fit_transform(dataset[columns_to_scale])

# verify that it worked correctly  --> "visual - inspection"
print(dataset[columns_to_scale].head())

In [ ]:
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/Look Up Tables/'
# Save the dataset
scaled_file_name = "scaled_allpatients.csv"
dataset.to_csv(f"{path}{scaled_file_name}", index=False)

In [ ]:
# look above to see which columns are selected
# safty check to see if i messed up
# more robust inspection --> mean should be close to 0 and standard deviation close to 1 --> all values have that so it looks good (:
print(dataset[columns_to_scale].mean())
print(dataset[columns_to_scale].std())